In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import sslearn
from sslearn import model_selection, wrapper
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.base import clone as skclone

In [2]:
import sklearn.datasets as data

In [3]:
X, y = data.load_iris(return_X_y=True)

In [4]:
resampling = resample(X, y, n_samples=int(X.shape[0]/3))

In [5]:
r1, r2 = resampling

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=float(1/3), random_state=0)

In [7]:
X_train.shape

(50, 4)

In [8]:
y_train

array([0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2,
       0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0,
       0, 0, 2, 1, 2, 0])

Prueba de Validación cruzada

In [10]:
skf = model_selection.StratifiedKFoldSS(n_splits=10, random_state=0, shuffle=True)
scores = []
scores_ind = []
for X_, y_, _, unlabel in skf.split(X_train,y_train):
    st = wrapper.DemocraticCoLearning(estimators=[skclone(DecisionTreeClassifier()) for _ in range(3) ])
    st.fit(X_,y_, [])
    scores.append(st.score(X_train[unlabel], y_train[unlabel]))
    scores_ind.append(st.score(X_test, y_test))
print("Precisión transductiva:",np.mean(scores))
print("Precisión inductiva:",np.mean(scores_ind))

TypeError: fit() missing 1 required positional argument: 'estimator_kwards'

Prueba creando un dataset artificial a partir de otro

In [70]:
X_, y_, true_label = model_selection.artificial_ssl_dataset(X_train, y_train, label_rate=0.10, random_state=0)
st = wrapper.CoForest(n_estimators=7, threshold=0.75)
score = st.fit(X_,y_).score(X_[y_ == -1], true_label)
score_ind = st.score(X_test, y_test)
print("Precisión transductiva:",score)
print("Precisión inductiva:",score_ind)

Precisión transductiva: 0.9333333333333333
Precisión inductiva: 0.93


In [71]:
X_, y_, true_label = model_selection.artificial_ssl_dataset(X_train, y_train, label_rate=0.05, random_state=0)
X_[y_==-1].shape, true_label.shape

((48, 4), (48,))

In [72]:
st.fit(X_,y_)
#.score(X_[y_ == -1], true_label)